# Importing the required libraries

In [ ]:
from airflow.models import DAG
from airflow.operators.bash_operator import BashOperator
from airflow.contrib.sensors.file_sensor import FileSensor
from airflow.operators.python_operator import PythonOperator
from datetime import datetime
from airflow.operators.bash_operator import BashOperator
from airflow.operators.email_operator import EmailOperator
from airflow.models import DAG
from airflow.operators.bash_operator import BashOperator
from airflow.contrib.sensors.file_sensor import FileSensor
from airflow.operators.python_operator import PythonOperator
from datetime import datetime
from airflow.operators.bash_operator import BashOperator
from airflow.operators.email_operator import EmailOperator
import pandas as pd
import json
import csv    
import psycopg2
from psycopg2 import Error
import subprocess

# Chekcing is Faker and MongoClient is installed in the airflow worker and schedular containers

In [ ]:
try:
    from faker import Faker
except:
    subprocess.check_call('pip'+ 'install' +'faker')
    from faker import Faker

try:
    from pymongo import MongoClient
except:
    subprocess.check_call('pip'+ 'install' +'pymongo')    
    from pymongo import MongoClient

# Building a function to generate dummy data using faker , the function will generate one csv file with 1000 record and place is in /opt/airflow/logs/
# This function will be called by a PythonOperator in the DAG

In [ ]:
###########################################Generating Dummy Data###########################################################################################################
def generate_dummy_data():    
    output=open('/opt/airflow/logs/data.CSV','w')
    fake=Faker()
    #Faker.seed(2020)
    header=['name','age','country','city','zipcode','state']
    mywriter=csv.writer(output)
    mywriter.writerow(header)
    for r in range(1000):
        mywriter.writerow([fake.name(),fake.random_int(min=18,max=80, step=1), fake.country(), fake.city(),fake.zipcode(),fake.state()])
    output.close()
    print("generate dummy data")

# Building a function to connect to the Postgres database and create a table called data_set with name,age , country , city , zipcode , and state 
# it will also read the generated CSV file from the previous step and dump it into the data_set table.
# The Postgres Database changes the IP Randomly when the container is up , it usually reserve one of the below IPs , but sometimes it change it to 172.18.0.5 ,if the flow failed trying to access the postgres on once of those IPs , then you will need to change it in this script to point to the right IP.
# This function will be called by a PythonOperator in the DAG

In [ ]:
###################################### Creating the table and loading the data into Postgres################################################################################

def load_into_postgres():


    try:
        connection = psycopg2.connect(user = "airflow",
                                  password = "airflow",
                                  host = "172.18.0.4",
                                  port = "5432",
                                  database = "postgres")
    except:
        connection = psycopg2.connect(user = "airflow",
                                  password = "airflow",
                                  host = "172.18.0.6",
                                  port = "5432",
                                  database = "postgres")
    cursor = connection.cursor()


    create_table_query = '''CREATE TABLE IF NOT EXISTS data_set
    (name  TEXT NOT NULL ,
    age  TEXT NOT NULL ,
    country  TEXT NOT NULL ,
    city TEXT NOT NULL ,
    zipcode TEXT NOT NULL ,
    state TEXT NOT NULL );'''

    cursor.execute(create_table_query)
    connection.commit()
    f = open(r'/opt/airflow/logs/data.CSV', 'r')
    print(f)
    print(cursor.copy_from(f, 'data_set', sep=','))
    connection.commit()
    f.close()
    connection.close()
    print("load the csv file into postgres database")

###################################### Extracting Data - Canada #########################################################################################################

def extract_from_postgres_canada():
    try:
        connection = psycopg2.connect(user = "airflow",
                                  password = "airflow",
                                  host = "172.18.0.4",
                                  port = "5432",
                                  database = "postgres")
    except:
        connection = psycopg2.connect(user = "airflow",
                                  password = "airflow",
                                  host = "172.18.0.6",
                                  port = "5432",
                                  database = "postgres")
    cursor = connection.cursor()
    sql = "COPY (SELECT * FROM data_set WHERE country = 'Canada') TO STDOUT WITH CSV HEADER "
    try:
        with open("/opt/airflow/logs/canada.csv", "w") as file:
          cursor.copy_expert(sql, file)
    except psycopg2.Error as e:
        print(e)
    connection.commit()
    cursor.close()
    connection.close()
    #print("extract the data and save it as csv")

# This function will be extracting data from the postgres database related to country= Brazil and store it in brazil.csv file .
# This function will be called by a PythonOperator in the DAG

In [ ]:
def extract_from_postgres_brazil():
    try:
        connection = psycopg2.connect(user = "airflow",
                                  password = "airflow",
                                  host = "172.18.0.4",
                                  port = "5432",
                                  database = "postgres")
    except:
        connection = psycopg2.connect(user = "airflow",
                                  password = "airflow",
                                  host = "172.18.0.6",
                                  port = "5432",
                                  database = "postgres")
    cursor = connection.cursor()
    sql = "COPY (SELECT * FROM data_set WHERE country = 'Brazil') TO STDOUT WITH CSV HEADER "
    try:
        with open("/opt/airflow/logs/brazil.csv", "w") as file:
          cursor.copy_expert(sql, file)
    except psycopg2.Error as e:
        print(e)
    connection.commit()
    cursor.close()
    connection.close()
    #print("extract the data and save it as csv")

# This function will be extracting data from the postgres database related to country= canada and store it in canada.csv file .
# This function will be called by a PythonOperator in the DAG

In [ ]:
def extract_from_postgres_canada():
    try:
        connection = psycopg2.connect(user = "airflow",
                                  password = "airflow",
                                  host = "172.18.0.4",
                                  port = "5432",
                                  database = "postgres")
    except:
        connection = psycopg2.connect(user = "airflow",
                                  password = "airflow",
                                  host = "172.18.0.6",
                                  port = "5432",
                                  database = "postgres")
    cursor = connection.cursor()
    sql = "COPY (SELECT * FROM data_set WHERE country = 'Canada') TO STDOUT WITH CSV HEADER "
    try:
        with open("/opt/airflow/logs/canada.csv", "w") as file:
          cursor.copy_expert(sql, file)
    except psycopg2.Error as e:
        print(e)
    connection.commit()
    cursor.close()
    connection.close()

# This function will be extracting data from the postgres database related to country= ghana and store it in ghana.csv file .
# This function will be called by a PythonOperator in the DAG

In [ ]:
def extract_from_postgres_ghana():
    try:
        connection = psycopg2.connect(user = "airflow",
                                  password = "airflow",
                                  host = "172.18.0.4",
                                  port = "5432",
                                  database = "postgres")
    except:
        connection = psycopg2.connect(user = "airflow",
                                  password = "airflow",
                                  host = "172.18.0.6",
                                  port = "5432",
                                  database = "postgres")
    cursor = connection.cursor()
    sql = "COPY (SELECT * FROM data_set WHERE country = 'Ghana') TO STDOUT WITH CSV HEADER "
    try:
        with open("/opt/airflow/logs/ghana.csv", "w") as file:
          cursor.copy_expert(sql, file)
    except psycopg2.Error as e:
        print(e)
    connection.commit()
    cursor.close()
    connection.close()
    #print("extract the data and save it as csv")


# This function will transform all the generated\extracted CSV files into Json files
# This function will be called by a PythonOperator in the DAG

In [ ]:
def csv_to_json():
    df = pd.read_csv("/opt/airflow/logs/canada.csv")
    df.to_json("/opt/airflow/logs/canada.json")
    print("transform the csv data to json for canada")
    df = pd.read_csv("/opt/airflow/logs/brazil.csv")
    df.to_json("/opt/airflow/logs/brazil.json")
    print("transform the csv data to json for brazil")
    df = pd.read_csv("/opt/airflow/logs/ghana.csv")
    df.to_json("/opt/airflow/logs/ghana.json")
    print("transform the csv data to json for ghana")

# This function will prepare the MongoDB and check if the database called Simple_Pipeline exisit or not
# This function will be called by a PythonOperator in the DAG

In [ ]:
def prepare_mongodb_db():
    client = MongoClient('mongo:27017',username='root', password='example')
    dblist = client.list_database_names()
    if "simple_pipline" in dblist:
        print("simple_pipline database exists")
        db = client['simple_pipeline']
    else:
        print("simple_pipline database is created now")
        db = client['simple_pipeline']

# This function will dump the canada.json document in the canada collection , as part of the Simple_Pipeline database on the MongoDB
# This function will be called by a PythonOperator in the DAG

In [ ]:
def load_json_into_mongo_canada():
    client = MongoClient('mongo:27017',username='root', password='example')
    db = client['simple_pipeline']
    collection_canada = db['Canada']
    
    with open('/opt/airflow/logs/canada.json') as f:
        file_data = json.load(f)
 
    collection_canada.insert_one(file_data)
    client.close()
    print("load the canada json file into the mongoDB")

# This function will dump the brazil.json document in the brazil collection , as part of the Simple_Pipeline database on the MongoDB
# This function will be called by a PythonOperator in the DAG

In [ ]:
def load_json_into_mongo_brazil():
    client = MongoClient('mongo:27017',username='root', password='example')
    db = client['simple_pipeline']
    collection_canada = db['Brazil']
    
    with open('/opt/airflow/logs/brazil.json') as f:
        file_data = json.load(f)
 
    collection_canada.insert_one(file_data)
    client.close()
    print("load the brazil json file into the mongoDB")

# This function will dump the ghana.json document in the ghana collection , as part of the Simple_Pipeline database on the MongoDB
# This function will be called by a PythonOperator in the DAG

In [ ]:
def load_json_into_mongo_ghana():
    client = MongoClient('mongo:27017',username='root', password='example')
    db = client['simple_pipeline']
    collection_canada = db['Ghana']
    
    with open('/opt/airflow/logs/ghana.json') as f:
        file_data = json.load(f)
 
    collection_canada.insert_one(file_data)
    client.close()
    print("load the ghana json file into the mongoDB")


# Building the DAG Object and call is Simple_Pipeline

In [ ]:
pipline_dag = DAG(
    dag_id = 'Simple_Pipline',
    #schedule_interval = "0 0 * * *",
    start_date=datetime(2021,4,5)
)


# Building the PythonOperators - Each one will call the designated callable function from the above defined ones.

In [ ]:
data_preparation_1 = PythonOperator(task_id='prepare_dummy_data',
                             python_callable=generate_dummy_data,
                             dag=pipline_dag)
                             
data_preparation_2 = PythonOperator(task_id='dump_into_postgres',
                             python_callable=load_into_postgres,
                             dag=pipline_dag)

data_extraction_canada = PythonOperator(task_id='extract_from_postgres_canada',
                             python_callable=extract_from_postgres_canada,
                             dag=pipline_dag)
data_extraction_brazil = PythonOperator(task_id='extract_from_postgres_brazil',
                             python_callable=extract_from_postgres_brazil,
                             dag=pipline_dag)
data_extraction_ghana = PythonOperator(task_id='extract_from_postgres_ghana',
                             python_callable=extract_from_postgres_ghana,
                             dag=pipline_dag)
data_transformation = PythonOperator(task_id='transform_data_structure',
                             python_callable=csv_to_json,
                             dag=pipline_dag)
prepare_warehouse = PythonOperator(task_id='prepare_warehouse',
                             python_callable=prepare_mongodb_db,
                             dag=pipline_dag)
data_load_canada = PythonOperator(task_id='load_data_canada',
                             python_callable=load_json_into_mongo_canada,
                             dag=pipline_dag)
data_load_brazil = PythonOperator(task_id='load_data_brazil',
                             python_callable=load_json_into_mongo_brazil,
                             dag=pipline_dag)
data_load_ghana = PythonOperator(task_id='load_data_ghana',
                             python_callable=load_json_into_mongo_ghana,
                             dag=pipline_dag)

## Finally , designing the flow of the pipeline 

In [ ]:
data_preparation_1 >> data_preparation_2 >> data_extraction_canada >> data_transformation >> prepare_warehouse
data_preparation_2 >> data_extraction_brazil >> data_transformation >> prepare_warehouse
data_preparation_2 >> data_extraction_ghana >> data_transformation >> prepare_warehouse
prepare_warehouse >> data_load_canada
prepare_warehouse >> data_load_brazil
prepare_warehouse >> data_load_ghana